# lasted edited by Claire Valva, May 23, 2018

# 1979 to 2016 300hPa at 40.5 deg data processing - Winter

Will look at winter spectra (Dec/Jan/Feb)

### file details
will be using file: 1979-2016-300hPa-40.5N-z.nc 
which is currently located in ~/uncategorized data

This file contains a single 2D array called “z”, whose dimension is (240, 55520).  The first dimension is longitude, 1.5 degree interval from 0 to 358.5 (0 is Greenwich Mean).  The second dimension is time, 6 hours apart, from 00 UTC January 1, 1979 to 18 UTC December 31, 2016.  The unit is meters^2/sec^2.  Divide by g = 9.81 m/s^2 to obtain geopotential height, the altitude at which you find 300 hPa.

In [1]:
#import packages, for now everything I think I'll need
import numpy as np
from netCDF4 import Dataset, num2date # This is to read .nc files and time array
from scipy.signal import get_window, csd
from scipy.fftpack import fft, ifft, fftshift, fftfreq
import matplotlib.pyplot as plt
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import pandas as pd
import datetime
from math import pi

In [2]:
filepath = '/home/clairev/uncategorized-data/1979-2016-300hPa-40.5N-z.nc' # Location of the file
fileobj = Dataset(filepath, mode='r')

# Check what's in there
fileobj

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): longitude(240), time(55520)
    variables(dimensions): float32 z(time,longitude)
    groups: 

In [16]:
#load coordinates
#so height[i] is the geopotential height at a given time
height = fileobj.variables['z'][:]/9.81

#create time array
time_list = []
for i in range(0,55520):
    time_list.append(i*6)
tunit = "hours since 1979-01-01T00:00:00Z"
tarray = num2date(time_list,units = tunit,calendar = 'gregorian')
print(tarray)

#create longitude array
lon_list = []
for i in range(240):
    lon_list.append(i*1.5)

[datetime.datetime(1979, 1, 1, 0, 0) datetime.datetime(1979, 1, 1, 6, 0)
 datetime.datetime(1979, 1, 1, 12, 0) ...,
 datetime.datetime(2016, 12, 31, 6, 0)
 datetime.datetime(2016, 12, 31, 12, 0)
 datetime.datetime(2016, 12, 31, 18, 0)]


In [4]:
#find and index the winter dates
winter_index = []

for i in range(0,55520):
    if tarray[i].month == 1:
        winter_index.append(i)
    if tarray[i].month == 2:
        winter_index.append(i)
    if tarray[i].month == 12:
        winter_index.append(i)

### apply zonal fft to all winter dates

In [20]:
#xf will be always the same
N = 240
T = 1./N
xfzone = fftfreq(N, d=(T))

In [45]:
#ck will change with date
ckzone_list = []

for i in winter_index: #change index to something smaller if need to check!
    snapshot = height[i]
    N = 240
    T = 1./N
    ck = 1/N * fft(snapshot)
    ckzone_list.append(ck)

In [54]:
#average ck over all ck! HAVE THE WRONG NUMBER!
avg_zonal = []

for i in range(N):
    newmean = np.mean(ckzone_list[:][i])
    avg_zonal.append(newmean)

### apply time-domain fft for winter dates (original code)

will no longer use? the code following this seems better, but I will leave the following blocks in markdown for reference

#choose a window length
window_length = 300.
freq_timedomain = [] #where each freq_timedomain[i] is time domain fft at longitude i*1.5
power_timedomain = [] #where power is indexed the same as freq_timedomain

#double index a list
for i in range(0,N): #range(0,N):
    loni = []
    
    for j in winter_index:
        loni.append(fileobj.variables['z'][j,i]/9.81)
    
    freqi, power_speci = csd(loni,loni, fs=4, 
                           window='hann', 
                           nperseg=window_length, scaling='spectrum')
    freq_timedomain.append(freqi)
    power_timedomain.append(power_speci)    

### alternate ways to apply hanning window
transform each year separately, and then take the average spectra

In [56]:
#create indexing for each year, so can find the dates
year_number = 2016 - 1979

year_index = [] #year_index[i] will be index of dates of year 1979 + i 

for j in range(0,year_number): #1979 left out because no december for that winter, as is 2016
    yearj_index = []
    for i in range(0,55520):
        if tarray[i].year == 1979 + j:
            yearj_index.append(i)
    year_index.append(yearj_index)

In [57]:
#find and index the winter dates by month
dec_index = []
jan_index = []
feb_index = []

for i in range(0,55520):
    if tarray[i].month == 1:
        jan_index.append(i)
    if tarray[i].month == 2:
        feb_index.append(i)
    if tarray[i].month == 12:
        dec_index.append(i)

In [ ]:
#the winter of the year, will be the winter that began in december of the year noted
freq_timebyyr = []
power_timebyyr = []

#choose a window length
window_length = 300

#then transform each year individually 
for i in range(0,N): #for each longitude
    freqi_byyr = [] #will append the fft for each year to this
    poweri_byyr = [] #will append the fft for each year to this, then add to the entire list
    
    for k in range(0,(year_number)): #go over each year - so can run and check other things
        
        lonik = []
        for j in range(0,55520):
            if j in dec_index and year_index[k]:
                lonik.append(fileobj.variables['z'][j,i]/9.81)
            
            if j in jan_index and year_index[k+1]:
                lonik.append(fileobj.variables['z'][j,i]/9.81)
            
            if j in feb_index and year_index[k + 1]:
                lonik.append(fileobj.variables['z'][j,i]/9.81)
    
        freqi_yeark, power_speci_yeark = csd(lonik,lonik, fs=4, 
                           window='hann', 
                           nperseg=window_length, scaling='spectrum')
        freqi_byyr.append(freqi_yeark)
        poweri_byyr.append(power_speci_yeark)

    freq_timebyyr.append(freqi_byyr)
    power_timebyyr.append(poweri_byyr)    

### take averages of the fft
freq = same for all of these, power = different, so will average all of the power with numpy mean function

In [ ]:
average_power = []
size_power = power_timebyyr[0][0].size

for i in range(size_power):
    meanpoweri = np.mean(power_timebyyr[:][:][i])
    average_power.append(meanpoweri)